In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform
import time, os, datetime
from Model import Residual_Unit
from Model import Attention_Block_NAL
from Model import AttentionResNet56_NAL

/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/jet/var/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: P

In [2]:
print("TF version: ",tf.__version__)
print("Keras version:",tf.keras.__version__)

TF version:  1.10.0
Keras version: 2.1.6-tf


In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,TensorBoard, LearningRateScheduler
from tensorflow.keras.models import load_model

In [4]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:45000, :, :, :]
y_train = y_train[:45000]
x_val = x_train[-5000:, :, :, :]
y_val = y_train[-5000:]

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (45000, 32, 32, 3)
y_train shape: (45000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [5]:
# define generators for training and validation data
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

# compute quantities required for feature normalization
train_datagen.fit(x_train)
val_datagen.fit(x_val)
test_datagen.fit(x_test)

In [ ]:
model = AttentionResNet56_NAL(shape=(32,32,3), in_channel=32, kernel_size=5, n_classes=10, dropout=None, regularization=0.0001)

def lr_schedule(epoch):
    lr = 1e-4
    if epoch > 80:
        lr *= 1e-5
    elif epoch > 60:
        lr *= 1e-4
    elif epoch > 40:
        lr *= 1e-2
    elif epoch >20:
        lr *= 1e-1
    print('Learning rate:', lr)
    return lr
lr_scheduler = LearningRateScheduler(lr_schedule)

# define loss, metrics, optimizer
optimizer = Adam(lr = lr_schedule(0) )
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Learning rate: 0.0001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   2432        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
batch_size = 128
epc = 100

start = time.time()

# training
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
step_size_train = train_generator.n // train_generator.batch_size

# validation
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size)
step_size_val = val_generator.n // val_generator.batch_size

# test
test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
step_size_test = test_generator.n // test_generator.batch_size

# usefull callbacks
log_dir="Logs/56full-NAL"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_reducer = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=5, verbose=1)
early_stopper = EarlyStopping(monitor='val_acc', patience=10, verbose=1)

model.fit_generator(train_generator,
                    steps_per_epoch = step_size_train,
                    epochs = epc,
                    validation_data = val_generator,
                    validation_steps = step_size_val,
                    callbacks=[tensorboard_callback, lr_reducer, lr_scheduler, early_stopper])

end = time.time()
print("Time taken by above cell is {}.".format((end-start)/60))

Learning rate: 0.0001
Epoch 1/100
351/351 [==============================] - 173s 494ms/step - loss: 1.8797 - acc: 0.3146 - val_loss: 1.7854 - val_acc: 0.3502
Learning rate: 0.0001
Epoch 2/100
351/351 [==============================] - 132s 377ms/step - loss: 1.6096 - acc: 0.4111 - val_loss: 1.8077 - val_acc: 0.4004
Learning rate: 0.0001
Epoch 3/100
351/351 [==============================] - 132s 377ms/step - loss: 1.4827 - acc: 0.4610 - val_loss: 1.8591 - val_acc: 0.4311
Learning rate: 0.0001
Epoch 4/100
351/351 [==============================] - 132s 376ms/step - loss: 1.3927 - acc: 0.4980 - val_loss: 1.3396 - val_acc: 0.5114
Learning rate: 0.0001
Epoch 5/100
351/351 [==============================] - 132s 375ms/step - loss: 1.3070 - acc: 0.5308 - val_loss: 1.2124 - val_acc: 0.5687
Learning rate: 0.0001
Epoch 6/100
351/351 [==============================] - 131s 374ms/step - loss: 1.2469 - acc: 0.5532 - val_loss: 1.1329 - val_acc: 0.5911
Learning rate: 0.0001
Epoch 7/100
351/351 [===

Learning rate: 1.0000000000000002e-06
Epoch 52/100
351/351 [==============================] - 131s 372ms/step - loss: 0.4120 - acc: 0.8556 - val_loss: 0.3169 - val_acc: 0.8858
Learning rate: 1.0000000000000002e-06
Epoch 53/100
351/351 [==============================] - 130s 370ms/step - loss: 0.4085 - acc: 0.8585 - val_loss: 0.3155 - val_acc: 0.8890
Learning rate: 1.0000000000000002e-06
Epoch 54/100
351/351 [==============================] - 131s 374ms/step - loss: 0.4146 - acc: 0.8557 - val_loss: 0.3176 - val_acc: 0.8868
Learning rate: 1.0000000000000002e-06
Epoch 55/100
351/351 [==============================] - 131s 373ms/step - loss: 0.4097 - acc: 0.8545 - val_loss: 0.3165 - val_acc: 0.8876
Learning rate: 1.0000000000000002e-06
Epoch 56/100
351/351 [==============================] - 132s 376ms/step - loss: 0.4077 - acc: 0.8559 - val_loss: 0.3153 - val_acc: 0.8876
Learning rate: 1.0000000000000002e-06
Epoch 57/100
351/351 [==============================] - 132s 376ms/step - loss: 0.

In [8]:
# evaluation
val_scores = model.evaluate_generator(val_generator, verbose=0)
test_scores = model.evaluate_generator(test_generator, verbose=0)
print('validation loss:', val_scores[0])
print('validation accuracy:', val_scores[1])
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

validation loss: 0.4210655170917511
validation accuracy: 0.854
Test loss: 0.7323221874713898
Test accuracy: 0.7592


In [0]:
model.save('Model56_paper.h5')  # 0.7418

In [11]:
mv 20191211-005015/ 56_NAL/

In [19]:
cd ..

/content/gdrive/My Drive/Deep-Learning
